# Hull-White + Time-Dependent Equity Volatility

This notebook illustrates how to:
1. **Bootstrap** a piecewise-constant equity volatility curve \(\sigma_S(t)\) from a set of market-implied volatilities at different maturities.
2. **Price** options under the single-factor Hull-White (Vasicek) short-rate model combined with a time-dependent equity volatility.

We assume:
- The short rate \(r(t)\) follows a one-factor Hull-White (Vasicek) SDE with mean reversion speed \(\alpha\) and volatility \(\sigma_r(t)\).
- The equity has **time-dependent volatility** \(\sigma_S(t)\) that we want to calibrate (or bootstrap) from market-implied volatilities.
- A **constant correlation** \(\rho\) between the short-rate Brownian motion and the equity Brownian motion.
- We switch to the **\(T\)-forward measure** so that the stock price at maturity is lognormal under these simplifying assumptions.

In [1]:
import sys
import os
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def B_HW(t, T, alpha):
    """
    Bond scaling factor for constant alpha in the Hull-White/Vasicek model:
    B(t,T) = (1 - exp(-alpha*(T - t))) / alpha
    """
    return (1.0 - np.exp(-alpha * (T - t))) / alpha

def integrated_variance(
    tgrid,      # array of time points, e.g. [0, ..., T]
    sigmaS,     # piecewise constant vol array matching intervals in tgrid
    alpha,      # mean reversion speed of short rate
    sigma_r,    # function sigma_r(t)
    B_func,     # function for B(t,T), e.g. B_HW
    rho         # correlation
):
    """
    Compute the integral of
       [sigma_S(t)^2 + 2 rho sigma_S(t)*B(t,T)*sigma_r(t) + (B(t,T)*sigma_r(t))^2]
    over t in [0, tgrid[-1]], using piecewise constant sigmaS(t) on sub-intervals.
    We'll do simple trapezoidal integration on each sub-interval [tgrid[i], tgrid[i+1]].
    """
    n = len(tgrid) - 1
    total = 0.0

    for i in range(n):
        t0 = tgrid[i]
        t1 = tgrid[i+1]
        sS = sigmaS[i]

        def integrand(t):
            Br = B_func(t, tgrid[-1], alpha)
            sr = sigma_r(t)
            return sS**2 + 2*rho*sS*Br*sr + (Br*sr)**2

        val0 = integrand(t0)
        val1 = integrand(t1)
        total += 0.5*(val0 + val1)*(t1 - t0)

    return total

def implied_vol(
    tgrid,
    sigmaS,     # piecewise constant vol array
    alpha,
    sigma_r,
    B_func,
    rho
):
    """
    Compute BS implied vol for maturity T = tgrid[-1]:
      sigma_imp^2 = (1/T) * ∫(...
    """
    T = tgrid[-1]
    var_T = integrated_variance(tgrid, sigmaS, alpha, sigma_r, B_func, rho)
    return math.sqrt(var_T / T)

def bs_call_price(F, K, T, vol):
    """
    Standard Black-Scholes call price on a forward F with strike K, maturity T, vol.
    """
    if T <= 0.0 or vol <= 0.0:
        return max(F - K, 0.0)
    d1 = (math.log(F/K) + 0.5*vol**2*T) / (vol*math.sqrt(T))
    d2 = d1 - vol*math.sqrt(T)
    return F*norm.cdf(d1) - K*norm.cdf(d2)

def option_price_hw_equity(
    S0,
    K,
    T,
    P0T,
    sigmaS,
    tgrid,
    alpha,
    sigma_r,
    B_func,
    rho
):
    """
    Price a European call under Hull-White + piecewise constant equity vol on [0..T].
    1) Compute the implied vol for maturity T.
    2) Price via standard Black-Scholes on forward F0 = S0 / P0T.
    3) Multiply by P0T to get present value.
    """
    volT = implied_vol(tgrid, sigmaS, alpha, sigma_r, B_func, rho)
    F0 = S0 / P0T
    call_bs = bs_call_price(F0, K, T, volT)
    return P0T * call_bs

def bootstrap_sigmaS(
    maturities,     # sorted array of maturities [T1, T2, ...]
    market_vols,    # corresponding implied vols
    alpha,
    sigma_r,
    B_func,
    rho,
    npts_per_interval=2
):
    """
    Given a set of market implied vols at maturities T1 < T2 < ... Tn,
    invert the formula:
      sigma_imp^2(T) * T = ∫(...) dt
    to solve for piecewise-constant sigmaS(t) on [0,T1], [T1,T2], ... .

    Returns:
      - tgrid_full: array of time points [0, T1, T2, ... Tn]
      - sigmaS_vals: piecewise-constant values for each sub-interval.
    """
    sigmaS_vals = []
    prev_T = 0.0
    tgrid_full = np.array([0.0])

    for i, T in enumerate(maturities):
        local_tgrid = np.linspace(prev_T, T, npts_per_interval)
        # merge with global tgrid
        if tgrid_full[-1] < prev_T:
            tgrid_full = np.concatenate([tgrid_full, local_tgrid])
        else:
            tgrid_full = np.concatenate([tgrid_full[:-1], local_tgrid])

        # target integrated variance at T
        target_var = market_vols[i]**2 * T

        def leftover_integral(x):
            """
            Construct a piecewise-constant sigmaS function from 0..T:
              - sigmaS_vals[j] for [T_{j-1}, T_j], j < i
              - x for [T_{i-1}, T_i]
            Then integrate.
            """
            seg_times = [0.0]
            seg_times.extend(maturities[:i])
            seg_times.append(T)
            seg_times = np.array(seg_times)

            seg_sigma = []
            for j in range(i):
                seg_sigma.append(sigmaS_vals[j])
            seg_sigma.append(x)

            # Build sub-partitions for integrated_variance
            sub_tgrid = []
            sub_sigma = []
            for k in range(len(seg_times)-1):
                sub_tgrid.append(seg_times[k])
                sub_sigma.append(seg_sigma[k])
            sub_tgrid.append(seg_times[-1])

            sub_tgrid = np.array(sub_tgrid)
            sub_sigma = np.array(sub_sigma)

            return integrated_variance(sub_tgrid, sub_sigma, alpha, sigma_r, B_func, rho)

        def obj(x):
            return leftover_integral(x) - target_var

        # Solve via bisection
        x_low, x_high = 1e-6, 3.0
        x_mid = 0.0
        for _ in range(60):
            x_mid = 0.5*(x_low + x_high)
            f_mid = obj(x_mid)
            if abs(f_mid) < 1e-12:
                break
            f_low = obj(x_low)
            if f_low*f_mid < 0:
                x_high = x_mid
            else:
                x_low = x_mid

        sigmaS_vals.append(x_mid)
        prev_T = T

    return tgrid_full, sigmaS_vals


## Example Usage

Below is a simple demonstration:
1. We have market data for maturities (1, 2, 3) years and implied vols of (20%, 22%, 25%).
2. We **bootstrap** the piecewise-constant equity volatilities.
3. We then **price** a sample call option with maturity 2.5 years and strike 100, given S0=100 and discount factor P(0,2.5)=0.9.

You can adapt these parts to fit your own data and usage.


In [3]:
# Example usage

# 1) Market data for maturities:
maturities = np.array([1.0, 2.0, 3.0])
#   Implied vols:
market_vols = np.array([0.20, 0.22, 0.25])  # 20%, 22%, 25%

# 2) Short-rate model parameters:
alpha = 0.1    # mean reversion speed
rho = 0.0      # correlation
def sigma_r_func(t):
    return 0.01  # short-rate vol = 1% (constant)

# 3) Bootstrap piecewise-constant sigmaS:
tgrid_boot, sigmaS_vals = bootstrap_sigmaS(
    maturities, market_vols,
    alpha, sigma_r_func,
    B_func=B_HW,
    rho=rho,
    npts_per_interval=2
)

print("Bootstrapped piecewise-constant equity volatilities:\n")
for i, T in enumerate(maturities):
    print(f" Interval [0..{T}] => sigmaS = {sigmaS_vals[i]:.4f}")

# 4) Price a call option (S0=100, K=100, T=2.5, discount factor=0.9):
S0 = 100.0
K  = 100.0
Tq = 2.5
P0T = 0.90

price_call = option_price_hw_equity(
    S0, K, Tq, P0T,
    sigmaS_vals,
    tgrid_boot,
    alpha,
    sigma_r_func,
    B_func=B_HW,
    rho=rho
)
print(f"\nOption price for maturity T={Tq}, strike K={K}: {price_call:.4f}")


Bootstrapped piecewise-constant equity volatilities:

 Interval [0..1.0] => sigmaS = 0.1999
 Interval [0..2.0] => sigmaS = 0.2379
 Interval [0..3.0] => sigmaS = 0.3003

Option price for maturity T=2.5, strike K=100.0: 20.4023
